# Hands-on example (Azure Resources)

In [ ]:
!python --version

In [2]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

#Azure
import azureml.core
from azureml.core import Workspace
from azure.storage.blob import BlobServiceClient

#MLFlow
import mlflow
import mlflow.sklearn
from mlflow.entities import ViewType

## 0. The data

* The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
* P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
* Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

In [3]:
data_path = "/Users/antfra/Desktop/MLFlow In Azure/data/wine-quality.csv" #put your path here

data = pd.read_csv(data_path)

data.sample(10)

/Users/antfra/.pyenv/versions/anaconda3-2019.10/envs/mlflow_tutorial/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
801,6.0,0.280,0.49,6.80,0.048,61.0,222.0,0.9953,3.19,0.47,9.3,5
147,6.4,0.595,0.14,5.20,0.058,15.0,97.0,0.9951,3.38,0.36,9.0,4
1287,8.3,0.210,0.40,1.60,0.032,35.0,110.0,0.9907,3.02,0.60,12.9,7
337,5.8,0.270,0.27,12.30,0.045,55.0,170.0,0.9972,3.28,0.42,9.3,6
1704,6.7,0.310,0.42,6.40,0.057,25.0,148.0,0.9955,3.16,0.45,9.6,5
2315,7.7,0.420,0.38,8.10,0.061,49.0,144.0,0.9966,3.40,0.58,11.0,6
3834,6.5,0.410,0.22,4.80,0.052,49.0,142.0,0.9946,3.14,0.62,9.2,5
2079,7.4,0.450,0.32,7.10,0.044,17.0,117.0,0.9962,3.32,0.41,10.4,4
1872,5.8,0.330,0.20,16.05,0.047,26.0,166.0,0.9976,3.09,0.46,8.9,5
100,7.4,0.250,0.37,13.50,0.060,52.0,192.0,0.9975,3.00,0.44,9.1,5


## 1. Tracking experiments

### Tracking stores
MLflow supports two types of backend stores: *file store* and *database-backed* store.

- Local file path (specified as file:/my/local/dir), where data is just directly stored locally. Defaults to `mlruns/`
- Database encoded as <dialect>+<driver>://<username>:<password>@<host>:<port>/<database>. Mlflow supports the dialects mysql, mssql, sqlite, and postgresql. For more details, see SQLAlchemy database uri.
- HTTP server (specified as https://my-server:5000), which is a server hosting an MLFlow tracking server.
- Databricks workspace (specified as databricks or as databricks://<profileName>, a Databricks CLI profile.
    
### Artifact stores
- Amazon S3
- Azure Blob Storage
- Google Cloud Storage
- FTP server
- SFTP Server
- NFS
- HDFS

Start the MLflow tracking server by 

```
mlflow server \
    --backend-store-uri postgresql+psycopg2://<username>:<password>@<host>:<port>/<database> \
    --default-artifact-root wasbs://<container>@<storage account>.blob.core.windows.net/<path> \
    --host 0.0.0.0
    --port 5000
```

or use the default storage method to write to `mlruns/`.

!mlflow server --backend-store-uri mlruns/ \
                --default-artifact-root wasbs://mlflow@robotdatastorage.blob.core.windows.net \
                --host 0.0.0.0 \
                --port 5000


In [8]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000
remote_server_uri = "http://0.0.0.0:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [13]:
mlflow.tracking.get_tracking_uri()

'http://0.0.0.0:5000'

### Choose Experiment

In [14]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

### What do we track?

- **Code Version**: Git commit hash used for the run (if it was run from an MLflow Project)
- **Start & End Time**: Start and end time of the run
- **Source**: what code run?
- **Parameters**: Key-value input parameters.
- **Metrics**: Key-value metrics, where the value is numeric (can be updated over the run)
- **Artifacts**: Output files in any format.

In [15]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5,full_view=False):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        if full_view:
            print("Run IDs: \n{}".format(mlflow.search_runs(ViewType.ACTIVE_ONLY)))
        else:
            pass
        
        mlflow.sklearn.log_model(lr, "model")

In [16]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.8222428497595401
  MAE: 0.6278761410160693
  R2: 0.12678721972772666
Save to: wasbs://mlflow@robotdatastorage.blob.core.windows.net/1/0ff576f7b401429f87b3685790e0a141/artifacts


In [17]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062342
  MAE: 0.6155290394093894
  R2: 0.20224631822892092
Save to: wasbs://mlflow@robotdatastorage.blob.core.windows.net/1/d596a979587a4f608e278de5d5e0e66d/artifacts


In [18]:
train(0.1, 0.1,full_view=True)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: wasbs://mlflow@robotdatastorage.blob.core.windows.net/1/c37fa1d72b204505b6350f76e2b01a6e/artifacts
Run IDs: 
                              run_id experiment_id    status  \
0   c37fa1d72b204505b6350f76e2b01a6e             1   RUNNING   
1   d596a979587a4f608e278de5d5e0e66d             1  FINISHED   
2   0ff576f7b401429f87b3685790e0a141             1  FINISHED   
3   f97f713e951b49f1bee62afed50f528f             1  FINISHED   
4   c98f6937f5e64fda997576ee686fe9b4             1    FAILED   
5   e2aded81d60143bb9af21c070ebbc358             1    FAILED   
6   bdb60f0f3fef4af580d30633598d7b40             1  FINISHED   
7   7ca8956a518a4fa5b3a63b1aa4bb8300             1  FINISHED   
8   0ad276658f1b480d8e3119df36da7185             1  FINISHED   
9   5cf6bf5f07af4694b5f8607d4a5f7013             1  FINISHED   
10  66261cd33e6d4864b2cc11b327922fc6          

### 1.1 Comparing runs
Run `mlflow ui` in a terminal or `http://your-tracking-server-host:5000` to view the experiment log and visualize and compare different runs and experiments. The logs and the model artifacts are saved in the `mlruns` directory (or where you specified).

## 2. Packaging the experiment as a MLflow project as conda env

Change directory to `mlproject`.

Specify the entrypoint for this project by creating a `MLproject` file and adding an conda environment with a `conda.yaml`. You can copy the yaml file from the experiment logs.

To run this project, invoke `mlflow run . -P alpha=0.42`. After running this command, MLflow runs your training code in a new Conda environment with the dependencies specified in `conda.yaml`.


## 3. Deploy the model

Deploy the model locally by running 

`mlflow models serve -m mlruns/0/f5f7c052ddc5469a852aa52c14cabdf1/artifacts/model/ -h 0.0.0.0 -p 1234`

Test the endpoint:

`curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"columns":["alcohol", "chlorides", "citric acid", "density", "fixed acidity", "free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],"data":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}' http://0.0.0.0:1234/invocations`

You can also simply build a docker image from your model

`mlflow models build-docker -m mlruns/1/d671f37a9c7f478989e67eb4ff4d1dac/artifacts/model/ -n elastic_net_wine`

and run the container with

`docker run -p 8080:8080 elastic_net_wine`.

Or you can directly deploy to Microsoft Azure ML.

## 4. Tagging runs

In [ ]:
from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.list_experiments() # returns a list of mlflow.entities.Experiment
print(experiments)

In [ ]:
# get the run
_run = client.get_run(run_id="aeffb800d7194a6f8472c03fcb5bbf5d")
print(_run)

In [14]:
# add a tag to the run
dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)